In [3]:
import os , pickle 
import pandas as pd

In [13]:
train_df = pd.read_excel("./dataset.xlsx",parse_dates=True)
train =   train_df.loc[train_df['pizza_name_id'] == 'bbq_ckn_l']
train

,order_date,formatted_date,pizza_name_id,quantity,unit_price,order_day,order_month,order_year,order_hour
40,2015-01-01,01 Jan 2015,bbq_ckn_l,1,20.75,Thursday,January,2015,18
56,2015-01-01,01 Jan 2015,bbq_ckn_l,1,20.75,Thursday,January,2015,22
78,2015-01-01,01 Jan 2015,bbq_ckn_l,1,20.75,Thursday,January,2015,15
85,2015-01-01,01 Jan 2015,bbq_ckn_l,1,20.75,Thursday,January,2015,13
130,2015-01-01,01 Jan 2015,bbq_ckn_l,1,20.75,Thursday,January,2015,15
...,...,...,...,...,...,...,...,...,...
48398,2015-12-30,30 Dec 2015,bbq_ckn_l,1,20.75,Wednesday,December,2015,20
48456,2015-12-31,31 Dec 2015,bbq_ckn_l,1,20.75,Thursday,December,2015,18
48463,2015-12-31,31 Dec 2015,bbq_ckn_l,1,20.75,Thursday,December,2015,18
48487,2015-12-31,31 Dec 2015,bbq_ckn_l,1,20.75,Thursday,December,2015,18


In [15]:
train_ = train.groupby(['order_date', 'pizza_name_id']).agg({'quantity': 'sum'}).reset_index()


    #valid_ = valid.groupby(['order_date', 'pizza_name_id']).agg({'quantity': 'sum'}).reset_index()


train_ = train_[['order_date', 'quantity']]

    #valid_ = valid_[['order_date', 'quantity']]
train_['order_date'] = pd.to_datetime(train_['order_date'])

In [19]:
train_

,order_date,quantity
0,2015-01-01,6
1,2015-01-02,4
2,2015-01-03,2
3,2015-01-05,2
4,2015-01-06,1
...,...,...
324,2015-12-26,3
325,2015-12-27,1
326,2015-12-29,1
327,2015-12-30,1


In [5]:
file_path = f"./Arima_models/"
file_name = f"Arima_bbq_ckn_l.pkl"  # Assuming you saved it as .pkl

# Full file path for loading the model
full_path = os.path.join(file_path, file_name)

# Load the ARIMA model
with open(full_path, 'rb') as f:
    model = pickle.load(f)

print(f"Model loaded from {full_path}")

Model loaded from ./Arima_models/Arima_bbq_ckn_l.pkl


In [7]:
start_date = '2016-01-01'
end_date = '2016-01-08'

date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create DataFrame
df_dates = pd.DataFrame(date_range, columns=['Date'])
df_dates

,Date
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
5,2016-01-06
6,2016-01-07
7,2016-01-08


In [33]:

pred = model.predict(start=len(train_), end=len(train_) + len(df_dates) -1, typ='levels').rename('ARIMA predictions')

# Output the predictions
print(pred)
p=list(pred)
res=sum(p)
print("ROUNDED----",round(res,0))

329    3.253651
330    3.013838
331    3.013838
332    3.013838
333    3.013838
334    3.013838
335    3.013838
336    3.013838
Name: ARIMA predictions, dtype: float64
ROUNDED---- 24.0


c:\Doughmate\my-env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Doughmate\my-env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Doughmate\my-env\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


In [27]:
index_future_dates =df_dates['Date']

pred.index=index_future_dates


In [28]:
pred


Date
2016-01-01    3.253651
2016-01-02    3.013838
2016-01-03    3.013838
2016-01-04    3.013838
2016-01-05    3.013838
2016-01-06    3.013838
2016-01-07    3.013838
2016-01-08    3.013838
Name: ARIMA predictions, dtype: float64

In [34]:
ingred = pd.read_excel("./Data/Pizza_ingredients.xlsx")
df = ingred[ingred['pizza_name_id'] == 'bbq_ckn_l']
df

,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0
5,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecue Sauce,15.0


In [35]:
for item in df['pizza_ingredients']:
    print(item)

Barbecued Chicken
Red Peppers
Green Peppers
Tomatoes
Red Onions
Barbecue Sauce
